![](../img/330-banner.png)

# Lecture 14: K-Means Clustering 

UBC 2022-23 W2

Instructor: Amir Abdi

### Announcements

- HW6 is due Mar 15, 11:59pm
- Hope you all learned something new in the midterm, btw, grades are out :)

## Imports, announcements, LOs

### Imports

You need to install the following package in the course environment.
```
conda install -c districtdatalabs yellowbrick
```

In [ ]:
# !conda install -c districtdatalabs yellowbrick -y

# If conda failed to install yellowbrick because of version conflict, try pip
# !pip install yellowbrick

In [ ]:
import os
import random
import sys
import time

import numpy as np

sys.path.append("../code/.")
import matplotlib.pyplot as plt
import mglearn
import seaborn as sns
from plotting_functions import *
from plotting_functions_unsup import *
from sklearn import cluster, datasets, metrics
from sklearn.compose import ColumnTransformer, make_column_transformer
from sklearn.datasets import make_blobs
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.preprocessing import StandardScaler
from yellowbrick.cluster import KElbowVisualizer, SilhouetteVisualizer
import pandas as pd

plt.style.use("seaborn")

%matplotlib inline

### Learning outcomes <a name="lo"></a>

From this lecture, students are expected to be able to:

- Explain the **unsupervised** paradigm. 
- Explain the motivation and potential applications of **clustering**. 
- Define the clustering problem. 
- Explain the **K-Means algorithm**.  
  - Apply `sklearn`'s `KMeans` algorithm.  
  - Point out pros and cons of K-Means and the difficulties associated with choosing the right number of clusters.   
  - Interpret the clusters discovered by K-Means. 
- Create the **Elbow plot** and **Silhouette plots** for a given dataset. 
- Visualize clusters in low dimensional space. 
- Use clustering for customer segmentation problem. 


<br><br><br><br>

## Introduction to unsupervised learning

### Types of machine learning

Here are some typical learning problems. 

- **Supervised learning**
    - Training a model from input data and and its corresponding targets (labels) to predict targets for new examples.     
- Self-supervised learning (SSL)
    - Similar to Supervised, but without any labels/targets, instead the input data (e.g. `X`) is the target.
- **Unsupervised learning**
    - Training a model to find patterns in a dataset, typically an unlabeled dataset
- Hybrid of Unsurpervised and Supervised
    - **SSL** training of Large Language Models (LLM) on general data followed by supervised training on downstream tasks
- Reinforcement learning
    - A family of algorithms for finding suitable actions to take in a given situation in order to maximize a reward. 


### Supervised learning

<img src="../img/sup-learning.png" height="1000" width="1000"> 

### Unsupervised learning

- Training data consists of observations ($X$) without any corresponding targets.
- Unsupervised learning could be used to group similar things together in $X$ or to find underlying structure in the data. 

<img src="../img/unsup-learning.png" alt="" height="900" width="900"> 

### Can we learn without targets?

- Yes, but the learning will be focused on finding the underlying structures of the inputs themselves (rather than finding the function $f$ between input and output like we did in supervised learning models). 

- Examples:
    - **Clustering** (this course)
    - Dimensionality reduction (not covered in this course; but, if you know about **Principal Component Analysis (PCA)**, you know enough)

### Labeled vs. Unlabeled data
- If you have access to labeled training data, you're in the "supervised" setting. 
- You know what to do in that case from the previous lectures
- **The amount of unlabeled data out there is much more than labeled data**
- Annotated data can become "stale" after a while in cases such as fraud detection. 
- Can you still make sense of the data even though you do not have the labels? 
  - Yes! At least to a certain extent! 

### Example: Supervised vs unsupervised learning

- In supervised learning, we are given features $X$ and target $y$. 


<table>
<tr style="background-color:white;">
    <td>
        <table>
            <tr>
                <td colspan="2" style="text-align:center;"> <b>Dataset 1</b> </td>
                <td></td>
            </tr>
            <tr>
                <td>$x_1$</td>
                <td>$y$</td>
            </tr>
            <tr>
                <td> 101.0
                <td> Sick
            </tr>
            <tr>
                <td> 98.5 
                <td> Not Sick
            </tr>
            <tr>
                <td> 93.8 
                <td> Sick
            </tr>
            <tr>
                <td> 104.3
                <td> Sick
            </tr>
            <tr>
                <td> 98.6 
                <td> Not Sick
            </tr>
        </table>
    </td>
    <td>
       <table>
            <tr>
                <td colspan="3" style="text-align:center;"> <b>Dataset2</b> </td>
                <td></td>
            </tr>
            <tr>
                <td>$x_1$</td>
                <td>$x_2$</td>
                <td>$y$</td>
            </tr>
            <tr>
                <td> -2.68
                <td> 0.32 
                <td>class 1
            </tr>
            <tr>
                <td> -2.71
                <td> -0.18
                <td> class 1
            </tr>
            <tr>
                <td> 1.28  
                <td> 0.69    
                <td> class 2
            </tr>
            <tr>
                <td> 0.93  
                <td> 0.32   
                <td> class 2
            </tr>
            <tr>
                <td> 1.39
                <td> -0.28 
                <td> class 3
            </tr>
        </table>
    </td>
</tr>
</table>

- In unsupervised learning, we are only given features $X$. 

<table>
<tr style="background-color:white;">
    <td>
        <table>
            <tr>
                <td colspan="2" style="text-align:center;"> <b>Dataset 1</b> </td>
                <td></td>
            </tr>
            <tr>
                <td>$x_1$</td>
            </tr>
            <tr>
                <td> 101.0
            </tr>
            <tr>
                <td> 98.5 
            </tr>
            <tr>
                <td> 93.8 
            </tr>
            <tr>
                <td> 104.3
            </tr>
            <tr>
                <td> 98.6 
            </tr>
        </table>
    </td>
    <td>
       <table>
            <tr>
                <td colspan="3" style="text-align:center;"> <b>Dataset 2</b> </td>
                <td></td>
            </tr>
            <tr>
                <td>$x_1$</td>
                <td>$x_2$</td>
            </tr>
            <tr>
                <td> -2.68
                <td> 0.32 
            </tr>
            <tr>
                <td> -2.71
                <td> -0.18
            </tr>
            <tr>
                <td> 1.28  
                <td> 0.69    
            </tr>
            <tr>
                <td> 0.93  
                <td> 0.32   
            </tr>
            <tr>
                <td> 1.39
                <td> -0.28 
            </tr>
        </table>
    </td>
</tr>
</table>

## Clustering motivation [[video](https://youtu.be/caAuUAXwpb8)]

### Clustering

- Suppose you are asked to categorize the items below. What groups would you identify? 


<img src="../img/food-clustering-activity.png" height="600" width="600"> 

Source: Images taken from different recipe sites on the internet.

- If you want to build a machine learning model to cluster such images how would you represent such images? 
- Do you think there is one correct way to cluster these images? 
- Imagine that we also have ratings data of food items and users for a large number of users. Can you exploit the power of community to recommend certain food items to a given user they are likely to consume? 

### Why clustering? 

- Most of the data out there is unlabeled.  
- Getting labeled training data is often difficult, expensive, or simply impossible in some cases. 
- Can we extract some useful information from unlabeled data? 
- The most intuitive way is to group similar examples together to get some insight into the data even though we do not have the targets.  

### Clustering  

**Clustering** is the task of partitioning the dataset into groups called clusters.

The goal of clustering is to discover underlying groups in a given dataset such that:
- **examples in the same group are as similar as possible**
- **examples in different groups are as different as possible**

### Input and possible output

In [ ]:
X, y = make_blobs(n_samples=10, centers=3, n_features=2, random_state=10)
fig, axes = plt.subplots(1, 2, figsize=(12, 4))
mglearn.discrete_scatter(X[:, 0], X[:, 1], markers="o", ax=axes[0])
mglearn.discrete_scatter(X[:, 0], X[:, 1], y, markers="o", ax=axes[1]);

Think of clustering as colouring the points (e.g., blue, red, green) such that points with the same color are close to each other. 

In [ ]:
mglearn.discrete_scatter(X[:, 0], X[:, 1], y, markers="o")
plt.legend();

- Usually the clusters are identified by a **cluster label**. 
- These labels are arbitrary, and **relabeling the points (label switching)** does not make a difference. 
- What we care about is which points have the same labels and which ones have different labels. 

### Is there a notion of "correct" grouping?

- Very often we do not know how many clusters are there in the data or if there are any clusters at all. In real-world data, clusters are rarely as clear as in our toy example above. 
- There is a notion of coherent and optimal (in some sense) clusters but there is no absolute truth here. 

### Example 1
Which of the following grouping of emoticons is the "correct" grouping?

![](../img/emoticon_clustering_example.png)

<!-- <img src="img/emoticon_clustering_example.png" alt="" height="800" width="800">  -->

Both seem reasonable! 

### Meaningful groups in clustering 
- In clustering, meaningful groups are dependent on the **application**.
- It usually helps if we have some prior knowledge about the data and the problem.   
- This makes it hard for us to objectively measure the quality of a clustering algorithm (or think about "true" clusters).

### Common applications: Data exploration

Although there is no notion of the "right" answer, we might still get something useful out of clustering. There are a number of common applications for clustering. 

- Summarize data 
- compress data
- Partition the data into groups before further processing. 
- You could use it in supervised learning setting as well: 
  - A) Carry out clustering and examine performance of your model on individual clusters. If the performance is lower on a particular cluster, you could either try building a separate model for that cluster and improve the overall performance of your supervised model
  - B) If you have few number of labeled samples, you can cluster the data, and use those few labeled samples and assume the same label for those within the same cluster.

### Common applications: Customer segmentation

- Understand landscape of the market in businesses and craft targeted business or marketing strategies tailored for each group.

<!-- ![](../img/customer-segmentation.png) -->

<img src="../img/customer-segmentation.png" alt="" height="600" width="600"> 

[source](https://www.youtube.com/watch?v=zPJtDohab-g&t=134s)

### Document clustering

Grouping articles on different topics from different news sources. For example, [Google News](https://news.google.com). 

![](../img/google_news.png)

<!-- <img src="img/google_news.png" alt="" height="1200" width="1200">  -->
    
**You'll be working on document clustering in the lab.**

### Similarity and distances

- Clustering is based on the **notion of similarity** or **distances** between points. 
- How do we determine similarity between points in a multi-dimensional space?
- Can we use something like $k$-neighbours for similarity? 
  - Yes! That's a good start!  
  - With $k$-neighbours we used Euclidean distances to find nearby points. 
  - We can use the same idea for clustering! 


<br><br><br><br><br><br>

## K-Means clustering algorithm [[video](https://youtu.be/s6AvSZ1_l7I)]

### K-Means clustering 

One of the most commonly used clustering algorithm. 

**Input**
- `X` $\rightarrow$ a set of data points  
- `K` (or $k$ or `n_clusters`) $\rightarrow$ **number of clusters** $\rightarrow$ Hyper-parameter

**Output**
- `K` clusters (groups) of the data points 



### K-Means using `sklearn`
- Before understanding the algorithm, let's try it with `sklearn`. 
- Consider the toy dataset above. 
- For this toy dataset, the three clusters are pretty clear.  

In [ ]:
X, y = make_blobs(n_samples=10, centers=3, n_features=2, random_state=10)
mglearn.discrete_scatter(X[:, 0], X[:, 1], markers="o");

In [ ]:
toy_df = pd.DataFrame(data=X, columns=["feat1", "feat2"])
toy_df

### `KMeans` `fit`
Let's try `sklearn`'s `KMeans` algorithm on this dataset.
- We need to decide how many clusters we want. Here we are passing 3. 
- We are only passing `X` because this is unsupervised learning; we do not have labels.  

In [ ]:
from sklearn.cluster import KMeans

# -------- New code -----------------
kmeans = KMeans(n_clusters=3)
kmeans.fit(X)


**We are only passing X because this is unsupervised learning**

### `predict` of `KMeans`

- The output of `KMeans` is $K$ clusters (groups) of the data points. 
- Calling `predict` on the **same train data** will give us the cluster assignment for each data point. 

In [ ]:
kmeans.predict(X)

In [ ]:
toy_df_cl = toy_df.copy()
toy_df_cl["cluster"] = kmeans.predict(toy_df)
toy_df_cl

### Cluster centers in  K-Means

- In K-Means each cluster is represented by its **cluster center**

In [ ]:
kmeans.cluster_centers_

There are **3 cluster centers**

In [ ]:
mglearn.discrete_scatter(X[:, 0], X[:, 1], kmeans.labels_, markers="o")
plt.legend()

# ---- visualize cluster centers ----------
mglearn.discrete_scatter(
    kmeans.cluster_centers_[:, 0], kmeans.cluster_centers_[:, 1], [0, 1, 2], markers="*"
);

### K-Means predictions on new examples 
- We can also use `predict` on unseen examples!  

In [ ]:
new_examples = np.array([[-1, -5], [2, 5.0]])
new_examples

In [ ]:
kmeans.predict(new_examples)

In [ ]:
mglearn.discrete_scatter(X[:, 0], X[:, 1], kmeans.labels_, markers="o")
plt.legend()
mglearn.discrete_scatter(
    kmeans.cluster_centers_[:, 0], kmeans.cluster_centers_[:, 1], [0, 1, 2], markers="*"
);

# ----- plotting new examples --------
mglearn.discrete_scatter(new_examples[:, 0], new_examples[:, 1], markers="^", s=15);

### K-Means algorithm: How does it work?

- Represent each cluster by its cluster center and assign a cluster membership to each data point. 

**Chicken-and-egg problem!**

- If we knew cluster centers, we can simply assign each point to its **nearest center**.
- Similarly, **if we knew assignments, we can calculate cluster centers**.  
- But we do not know either 😟. 

<br><br><br><br><br>
**Expectation Maximization** is a family of **iterative algorithms** in Computer Science, and K-Means is a special form of that.
<br><br><br><br><br>

### K-Means clustering algorithm

**Input**: Data points X and the number of clusters K

**Initialization**: K **random initial centers** for the clusters

**Iterative process**:

repeat 
- Assign each example to the closest center.
- Estimate new centers as _average_ of observations in a cluster.

until **centers stop changing** or **maximum iterations have reached**.

<br><br><br><br>

Try this **Interactive Demo** to learn about K-Means: http://alekseynp.com/viz/k-means.html

<br><br><br><br><br>

-----------

[Review these cells on your own]

## Non-interactive Demo of K-Means

Let's execute K-Means algorithm on our toy example. 

**Input**
- The data points `X`

In [ ]:
n_examples = toy_df.shape[0]
print("Number of examples: ", n_examples)
X

- Let K (number of clusters) be 3. 

In [ ]:
k = 3

### Initialization 

- Random initialization for K initial centers of the clusters. 

In [ ]:
np.random.seed(seed=14)
centers_idx = np.random.choice(range(0, n_examples), size=k)
centers_df = toy_df.iloc[centers_idx]
centers = X[centers_idx]
colours = ["black", "blue", "red"]

In [ ]:
plt.scatter(X[:, 0], X[:, 1], marker="o")
plt.scatter(centers[:, 0], centers[:, 1], c=colours, marker="*", s=200)
plt.title("Initial centers");

### Iterative process

repeat 

- Assign each example to the closest center. (`update_Z`)
- Estimate new centers as _average_ of observations in a cluster. (`update_centers`)

until **centers stop changing** or **maximum iterations have reached**.

### How to find closest centers? 

- First step in the iterative process is assigning examples to the closest center. 
- Let's consider distance of an example to all centers and assign that example to the closest center.  

In [ ]:
plot_example_dist(toy_df, centers_df, 6, 4)

### How to find closest centers?

- Similarly, we can make cluster assignments for all points by calculating distances of all examples to the centers and assigning it to the cluster with smallest distance.  

In [ ]:
from sklearn.metrics import euclidean_distances


def update_Z(X, centers):
    """
    returns distances and updated cluster assignments
    """
    dist = euclidean_distances(X, centers)
    return dist, np.argmin(dist, axis=1)

### How to update centers?   

- With the new cluster assignments for our data points, we update cluster centers. 
- New cluster centers are means of data points in each cluster. 

In [ ]:
def update_centers(X, Z, old_centers, k):
    """
    returns new centers
    """
    new_centers = old_centers.copy()
    for kk in range(k):
        new_centers[kk] = np.mean(X[Z == kk], axis=0)
    return new_centers

### Iteration 1: Step 1 

- Assign each example to the closest cluster center.

In [ ]:
dist, Z = update_Z(X, centers)
Z

- This is the current cluster assignment. 

In [ ]:
plot_current_assinment(X, Z, centers)

### Iteration 1: Step 2 
- Estimate new centers as _average_ of observations in a cluster.

In [ ]:
new_centers_it1 = update_centers(X, Z, centers, k)

- This is how the centers moved in this iteration. 

In [ ]:
plot_update_centroid(toy_df, 6, 4, new_centers_it1, centers, dist)

### Iteration 2: step 1

- Assign each example to the closest cluster center.

In [ ]:
dist, Z = update_Z(X, new_centers_it1)
Z

- This is the current cluster assignment. 

In [ ]:
plot_current_assinment(X, Z, new_centers_it1)

### Iteration 2: step 2

- Estimate new centers as _average_ of observations in a cluster.

In [ ]:
new_centers_it2 = update_centers(X, Z, new_centers_it1, k)

- This is how the centers moved in this iteration. 

In [ ]:
plot_update_centroid(toy_df, 6, 4, new_centers_it2, new_centers_it1, dist)

### Iteration 3: step 1

- Assign each example to the closest cluster center.

In [ ]:
dist, Z = update_Z(X, new_centers_it2)
Z

- This is the current cluster assignment. 

In [ ]:
plot_current_assinment(X, Z, new_centers_it2)

### Iteration 3: step 2

- Estimate new centers as _average_ of observations in a cluster.

In [ ]:
new_centers_it3 = update_centers(X, Z, new_centers_it2, k)

- This is how the centers moved in this iteration. 

In [ ]:
plot_update_centroid(toy_df, 6, 4, new_centers_it3, new_centers_it2, dist)

### Iteration 4: step 1

- Assign each example to the closest cluster center.

In [ ]:
dist, Z = update_Z(X, new_centers_it3)
Z

- This is the current cluster assignment. 

In [ ]:
plot_current_assinment(X, Z, new_centers_it3)

### Iteration 4: step 2

- Estimate new centers as _average_ of observations in a cluster.

In [ ]:
new_centers_it4 = update_centers(X, Z, new_centers_it3, k)

- The cluster centers are not moving anymore. 

In [ ]:
plot_update_centroid(toy_df, 6, 4, new_centers_it4, new_centers_it3, dist)

### Iteration 5: step 1

- Assign each example to the closest cluster center.

In [ ]:
dist, Z = update_Z(X, new_centers_it4)
Z

- This is the current cluster assignment.

In [ ]:
plot_current_assinment(X, Z, new_centers_it4)

### Iteration 5: step 2

- Estimate new centers as _average_ of observations in a cluster.

In [ ]:
new_centers_it5 = update_centers(X, Z, new_centers_it4, k)

- The cluster centers are not moving anymore. 

In [ ]:
plot_update_centroid(toy_df, 6, 4, new_centers_it5, new_centers_it4, dist)

### When to stop?

- Seems like our centroids aren't changing anymore. 
- The algorithm has converged. So we stop! 
- K-Means always converges. It doesn't mean it finds the "right" clusters. It can converge to a sub-optimal solution.   

In [ ]:
plot_iterative(toy_df, 6, 4, centers)

Initialization is crucial. We'll talk about it in a bit. 

[End of non-interactive demo]

-----------
<br><br><br><br><br><br>

### Example 2
- Let's use the K-means on the iris dataset. 

In [ ]:
## Iris dataset
iris = datasets.load_iris()  # loading the iris dataset
features = iris.data  # get the input data
labels = iris.target_names[
    iris.target
]  # get the targets, in this case the types of the Iris flower

iris_df = pd.DataFrame(features, columns=iris.feature_names)
iris_df

In [ ]:
np.unique(labels)

Iris has 4 features. We cannot visualize 4 features, so, we will use Principal Component Analysis (PCA) to reduce the dimensions to 2.


**Wait, what is PCA?**
- It reducs dimensionality (number of features) of the data in a way that explains the **most variance** in the data.
- It is also used as an Exploratory Data Analysis (EDA) tool

**How does PCA work?**
- Learn the following if you are interested: Eigenvectors / Eigenvalues / Covariance Matrix

Note: *We might come back to PCA once again in Recommendation Systems*

In [ ]:
# Reducing the dimensionality for plotting purposes

# ------ new code ---------
pca = PCA(n_components=2)
pca.fit(features)
# ----------------------
data_iris = pd.DataFrame(pca.transform(features), columns=["$Z_1$", "$Z_2$"])
data_iris["target"] = labels

**Here, distance (similarty) is Euclidean**

In [ ]:
plot_unsup(data_iris, 20, 10, "Iris dataset")

#### Initialization

- In this case, **we know that $k=3$**;
  - If not, we might have aimed for 2 clusters
- We are going to pick three points at random to use as initial centroids;

In [ ]:
# RANDOM initialization
k = 3
centroids = np.random.choice(range(0, 150), size=k)
centroids = data_iris.iloc[centroids, 0:2]
plot_intial_center(data_iris, centroids, 22, 10, title="Iris dataset")

- Next, for each point in the dataset, we calculate the distance to each one of the centroids; 


- Let's do it for one point as example:

In [ ]:
plot_example_dist(data_iris, centroids, 22, 10)

In [ ]:
dist = distance.cdist(data_iris.iloc[:, 0:2], centroids.iloc[:, 0:2])
plot_first_assignment(data_iris, centroids, dist, 22, 10)

In [ ]:
plot_iterative(data_iris, 22, 10, centroids.to_numpy())

And we iterate until we get **converge**...

<br><br><br><br><br>

### (Optional) Feature engineering using K-Means 

- K-Means could be used for feature engineering in supervised learning. 
- Examples: 
    - You could add a categorical feature: cluster membership
    - You could add a continuous features: distance from each cluster center
- See [this paper](http://ai.stanford.edu/~acoates/papers/coatesleeng_aistats_2011.pdf).

<br><br><br><br>

## Choosing K [[video](https://youtu.be/M5ilrhcL0oY)]

### Hyperparameter tuning for K

- K-Means takes K (`n_clusters` in `sklearn`) as a hyperparameter. How do we pick K? 

- In supervised setting we carried out hyperparameter optimization based on cross-validation scores. 

- Since in unsupervised learning we do not have the target values, it becomes difficult to **objectively measure the effectiveness of the algorithms**.

- There is no definitive approach.

- However, some strategies might be useful to help you determine K. 

### Method 1: The Elbow method

- This method looks at the sum of **intra-cluster distances**, which is also referred to as **inertia**. 
- The intra-cluster distance in our toy example above is given as   

$$\sum_{P_i \in C_1}  distance(P_i, C_1)^2 + \sum_{P_i \in C_2}  distance(P_i, C_2)^2 + \sum_{P_i \in C_3} distance(P_i, C_3)^2$$

Where 
- $C_1, C_2, C_3$ are centroids 
- $P_i$s are points within that cluster
- $distance$ can be any distance, depending on the problem (e.g. Euclidean distance)

### Inertia 

You can access this intra-cluster distance or inertia as follows. 

In [ ]:
X, y = make_blobs(centers=3, n_features=2, random_state=10)
mglearn.discrete_scatter(X[:, 0], X[:, 1], markers="o");

If we iterate over different number of **K values** and calcualte the **inertia** for each:

In [ ]:
d = {"K": [], "inertia": []}
for k in range(1, 100, 10):
    model = KMeans(n_clusters=k).fit(X)
    d["K"].append(k)
    d["inertia"].append(model.inertia_)

In [ ]:
pd.DataFrame(d)

- The **inertia decreases as K increases**. 
- Question: Do we want inertia to be small or large? 
  - Answer: ?????

- The problem is that we can't just look for a $k$ that minimizes inertia because it decreases as $k$ increases.
    - If I have number of clusters = number of examples, each example will have its own cluster and the intra-cluster distance will be 0.  
- Instead we evaluate the trade-off between **two objectives**:
  - "small k"
  - "small intra-cluster distances". 

In [ ]:
def plot_elbow(w, h, inertia_values):
    plt.figure(figsize=(w, h))
    plt.axvline(x=3, linestyle="-.", c="black")
    plt.plot(range(1, 10), inertia_values, "-o")
    ax = plt.gca()
    ax.tick_params("both", labelsize=(w + h) / 2)
    ax.set_xlabel("K", fontsize=w)
    ax.set_ylabel("Inertia", fontsize=w)

In [ ]:
inertia_values = list()
for k in range(1, 10):
    inertia_values.append(KMeans(n_clusters=k).fit(toy_df).inertia_)
plot_elbow(8, 6, inertia_values)

- From the above plot, we could argue that **3 clusters** are enough.
- The inertia decreases when clusters are greater than 3. However it's not a big improvement and so we prefer K=3. 
- In this toy example, it's the plot is kind of clear and easy to interpret but it can be hard to interpret in real life examples. 

<br><br><br><br><br>

There  package [`yellowbrick`](https://www.scikit-yb.org/en/latest/api/cluster/elbow.html) which can be used to create these plots conveniently. 

Check documentation of `KElbowVisualizer` here for KMeans: https://www.scikit-yb.org/en/latest/api/cluster/elbow.html

In [ ]:
from yellowbrick.cluster import KElbowVisualizer

model = KMeans()

# ----- pass the model and HParams to KElbowVisualizer ------
visualizer = KElbowVisualizer(model, k=(1, 10))

visualizer.fit(X)  # Fit the data to the visualizer
visualizer.show();

<br><br>

### Method 2: The Silhouette method

- Not dependent on the notion of cluster centers. 
- Calculated using the **mean intra-cluster distance** ($a$) and the **mean nearest-cluster distance** ($b$) for each sample.

### Mean intra-cluster distance ($a$)

- Suppose the green point below is our sample. 
- Average of the distances of the green point to the other points in the same cluster.
  - These distances are represented by the black lines. 
  

### Mean nearest-cluster distance ($b$)

- Average of the distances of the green point to the blue points is smaller than the average of the distances of the green point to the red points. So the **nearest cluster** is the blue cluster. 
- So the mean nearest-cluster distance is the average of the distances of the green point to the blue points.  

In [ ]:
plot_silhouette_dist(6, 4)

### Silhouette distance for a sample 

- the difference between the **the average nearest-cluster distance** ($b$) and **average intra-cluster distance** ($a$) for each sample, normalized by the maximum value

$$\frac{b-a}{max(a,b)}$$


In theory:
- The **best** value is 1. 
- The worst value is -1 (samples have been assigned to wrong clusters).
- Value near 0 means overlapping clusters. 

The overall **Silhouette score** is the average of the Silhouette scores for all samples. 

### Using Silhouette scores to select the number of clusters

- The plots below show the Silhouette scores for each sample in that cluster. 
- Higher values indicate well-separated clusters. 
- The size of the Silhouette shows the number of samples and hence shows imbalance of data points in clusters.  

In [ ]:
from yellowbrick.cluster import SilhouetteVisualizer

In [ ]:
model = KMeans(2, random_state=42)
visualizer = SilhouetteVisualizer(model, colors="yellowbrick")
visualizer.fit(X)  # Fit the data to the visualizer
visualizer.show();
# Finalize and render the figure

In [ ]:
model = KMeans(3, random_state=42)
visualizer = SilhouetteVisualizer(model, colors="yellowbrick")
visualizer.fit(X)  # Fit the data to the visualizer
visualizer.show();
# Finalize and render the figure

In [ ]:
model = KMeans(5, random_state=42)
visualizer = SilhouetteVisualizer(model, colors="yellowbrick")
visualizer.fit(X)  # Fit the data to the visualizer
visualizer.show();
# Finalize and render the figure

### What to look for in these plots?

- Unlike inertia, **larger values are better** because they indicate that the point is further away from neighbouring clusters.
- The thickness of each silhouette indicates the cluster size.
- The shape of each silhouette indicates the "goodness" for points in each cluster.
  - **A slower dropoff (more rectangular)** indicates more points are "happy" in their cluster.
  - **A fast drop** shows that some points are not very aligned within their cluster.
  - **Negative values** are red flags and indicate something has gone terribly bad.
- **The length (or area) of each silhouette indicates the goodness of each cluster.**


In [ ]:
model = KMeans(3, random_state=42)
visualizer = SilhouetteVisualizer(model, colors="yellowbrick")
visualizer.fit(X)  # Fit the data to the visualizer
visualizer.show();
# Finalize and render the figure

### Comments on Silhouette scores 
- Unlike inertia, larger values are better because they indicate that the point is further away from neighbouring clusters.
- Unlike inertia, **the overall silhouette score gets worse as you add more clusters because you end up being closer to neighbouring clusters.**
- Thus, as with intertia, you will not see a "peak" value of this metric that indicates the best number of clusters.
- We can visualize the silhouette score for each example individually in a silhouette plot (hence the name), see below.
- We can apply Silhouette method to clustering methods other than K-Means. 


<br><br><br><br>

### ❓❓ Questions for you

### (iClicker) Exercise 14.1 

**iClicker cloud join link: https://join.iclicker.com/EMMJ**

**Select all of the following statements which are TRUE.**

- (A) K-Means may converge to different solutions depending upon the initialization.
- (B) K-means terminates when the number of clusters does not increase between iterations.
- (C) $K$ in K-Means should always be $\leq$ # of features.
- (D) In K-Means, it makes sense to have $K$ $<$ # of examples. 
- (E) In K-Means, in some iterations some points may be left unassigned. 

<br><br><br><br>

### (iClicker) Exercise 14.2

**iClicker cloud join link: https://join.iclicker.com/EMMJ**

**Select all of the following statements which are TRUE.**

- (A) The preprocessing methods such as `StandardScaler` are unsupervised methods. 
- (B) If the centroid locations do not change between iterations, K-means terminates.
- (C) If you train K-Means with `n_clusters= the number of examples`, the inertia value will be 0. 
- (D) Unlike the Elbow method, the Silhouette method is not dependent on the notion of cluster centers.    

<br><br><br><br>

## K-Means case study: Customer segmentation

### What is customer segmentation? 

- Understand landscape of the market in businesses and craft targeted business or marketing strategies tailored for each group.

<img src="../img/customer-segmentation.png" alt="" height="600" width="600"> 

[source](https://www.youtube.com/watch?v=zPJtDohab-g&t=134s)

Check out [this interesting talk by Malcom Gladwell](https://www.ted.com/talks/malcolm_gladwell_on_spaghetti_sauce?language=en). Humans are diverse and there is no single spaghetti sauce that would make all of them happy! 

Often it's beneficial to businesses to explore the landscape of the market and tailor their services and products offered to each group. This is called **customer segmentation**. It's usually applied when the dataset contains some of the following features. 

- **Demographic information** such as gender, age, marital status, income, education, and occupation
- **Geographical information** such as specific towns or counties or a customer's city, state, or even country of residence (in case of big global companies)
- **Psychographics** such as social class, lifestyle, and personality traits
- **Behavioral data** such as spending and consumption habits, product/service usage, and desired benefits 

### Business problem 

- Imagine that you are hired as a data scientist at a bank. They provide some data of their credit card customers to you. 
- Their goal is to develop customized marketing campaigns and they ask you to group customers based on the given information. 
- Now that you know about K-Means clustering, let's apply it to the dataset to group customers. 

### Data

- We will use the [Credit Card Dataset for clustering](https://www.kaggle.com/arjunbhasin2013/ccdata) from Kaggle.
- Download the data and save the CSV under the `data` folder. 
- I encourage you to work through this case study on your own. 

In [ ]:
creditcard_df = pd.read_csv("../data/CC General.csv")
creditcard_df.shape

In [ ]:
creditcard_df

### Information of the dataset 

We have behavioral data. 

- CUSTID: Identification of Credit Card holder
- BALANCE: Balance amount left in customer's account to make purchases
- BALANCE_FREQUENCY: How frequently the Balance is updated, score between 0 and 1 (1 = frequently updated, 0 = not frequently updated)
- PURCHASES: Amount of purchases made from account
- ONEOFFPURCHASES: Maximum purchase amount done in one-go
- INSTALLMENTS_PURCHASES: Amount of purchase done in installment
- CASH_ADVANCE: Cash in advance given by the user
- PURCHASES_FREQUENCY: How frequently the Purchases are being made, score between 0 and 1 (1 = frequently purchased, 0 = not frequently purchased)
- ONEOFF_PURCHASES_FREQUENCY: How frequently Purchases are happening in one-go (1 = frequently purchased, 0 = not frequently purchased)
- PURCHASES_INSTALLMENTS_FREQUENCY: How frequently purchases in installments are being done (1 = frequently done, 0 = not frequently done)
- CASH_ADVANCE_FREQUENCY: How frequently the cash in advance being paid
- CASH_ADVANCE_TRX: Number of Transactions made with "Cash in Advance"
- PURCHASES_TRX: Number of purchase transactions made
- CREDIT_LIMIT: Limit of Credit Card for user
- PAYMENTS: Amount of Payment done by user
- MINIMUM_PAYMENTS: Minimum amount of payments made by user
- PRC_FULL_PAYMENT: Percent of full payment paid by user
- TENURE: Tenure of credit card service for user

### Preliminary EDA

In [ ]:
creditcard_df.info()

- All numeric features
- Some missing values

In [ ]:
creditcard_df.describe()

### Practice exercises for you

1. What is the average `BALANCE` amount?
2. How often the `BALANCE_FREQUENCY` is updated on average? 
3. Obtain the row the customer who made the maximum cash advance transaction. 

<br><br><br><br>

### Mini exercises for you (Answers)

1. What is the average `BALANCE` amount? 1564.47
2. How often the `BALANCE_FREQUENCY` is updated on average? 0.877 (pretty often) 
3. Obtain the row of the customer who made the maximum cash advance transaction. 

In [ ]:
# Answer 3.
max_cash_advance = creditcard_df["CASH_ADVANCE"].max()
creditcard_df[creditcard_df["CASH_ADVANCE"] == max_cash_advance]

Let's examine correlations between features. 

In [ ]:
cor = creditcard_df.corr()
plt.figure(figsize=(20, 10))
sns.set(font_scale=1)
sns.heatmap(cor, annot=True, cmap=plt.cm.Blues);

### Feature types and preprocessing 

Let's identify different feature types and transformations 

In [ ]:
creditcard_df.columns

In [ ]:
drop_features = ["CUST_ID"]
numeric_features = list(set(creditcard_df.columns) - set(drop_features))

- What kind of preprocessing do we need to apply? 
- Do we need to scale the data? 

In [ ]:
from sklearn.impute import SimpleImputer

numeric_transformer = make_pipeline(SimpleImputer(), StandardScaler())

preprocessor = make_column_transformer(
    (numeric_transformer, numeric_features), ("drop", drop_features)
)

In [ ]:
transformed_df = pd.DataFrame(
    data=preprocessor.fit_transform(creditcard_df), columns=numeric_features
)

In [ ]:
transformed_df

Now that we have transformed the data, we are ready to run K-Means to cluster credit card customers. 

### Choosing `n_clusters`

- There is no definitive method to find the optimal number of clusters. 
- Let's try different approaches. 

### The Elbow method

In [ ]:
model = KMeans(random_state=42)
visualizer = KElbowVisualizer(model, k=(1, 20))

visualizer.fit(transformed_df)  # Fit the data to the visualizer
visualizer.show();

- The optimal number of clusters is not as clear as it was in our toy example. 

- Let's examine Silhouette scores.  

In [ ]:
for k in range(3, 6):
    model = KMeans(k, random_state=42)
    visualizer = SilhouetteVisualizer(model, colors="yellowbrick")
    visualizer.fit(transformed_df)  # Fit the data to the visualizer
    visualizer.show()

- I'm going to run `KMeans` with `n_clusters = 4`. 
- You can try out `n_clusters = 5` and `n_clusters = 6` on your own. 

### Visualizing clusters 

- Can we visualize the clusters? 
- We have a high dimensional data and we need to reduce the dimensionality in order to visualize it. 
- Let's reduce the dimensionality using a technique called [UMAP](https://umap-learn.readthedocs.io/en/latest/index.html). 

I forgot to put this package in the course environment file. So to run the code below, you'll have to install the `umap-learn` package in the course conda environment either with `conda` or `pip`, as described in the [documentation](https://umap-learn.readthedocs.io/en/latest/index.html). 

```
> conda install -c conda-forge umap-learn
```

or 

```
> pip install umap-learn
```


In [ ]:
# !pip install umap-learn

In [ ]:
import umap

In [ ]:
def plot_umap_clusters(
    data,
    cluster_labels,
    size=50,
    n_neighbors=15,
    title="UMAP visualization",
):
    """
    Carry out dimensionality reduction using UMAP and plot 2-dimensional clusters.

    Parameters
    -----------
    data : numpy array
        data as a numpy array
    cluster_labels : list
        cluster labels for each row in the dataset
    size : int
        size of points in the scatterplot
    n_neighbors : int
        n_neighbors hyperparameter of UMAP. See the documentation.
    title : str
        title for the visualization plot

    Returns
    -----------
    None. Shows the clusters.
    """

    reducer = umap.UMAP(n_neighbors=n_neighbors)
    Z = reducer.fit_transform(data)  # reduce dimensionality
    umap_df = pd.DataFrame(data=Z, columns=["dim1", "dim2"])
    umap_df["cluster"] = cluster_labels

    labels = np.unique(umap_df["cluster"])

    fig, ax = plt.subplots(figsize=(10, 7))
    ax.set_title(title)

    scatter = ax.scatter(
        umap_df["dim1"],
        umap_df["dim2"],
        c=umap_df["cluster"],
        cmap="tab20b",
        s=size,
        edgecolors="k",
        linewidths=0.1,
    )

    legend = ax.legend(*scatter.legend_elements(), loc="best", title="Clusters")
    ax.add_artist(legend)

    plt.show()

In [ ]:
for k in range(3, 7):
    kmeans = KMeans(n_clusters=k, random_state=42)
    kmeans.fit(transformed_df)
    labels = kmeans.labels_
    plot_umap_clusters(transformed_df, kmeans.labels_, title=f"K-Means with k = {k}")

- The clusters above look reasonably well separated. 
- This might not always be the case. 

### Cluster interpretation

- Let's examine the cluster centers for k=4 and identify types of customers.  

In [ ]:
reasonable_k = 4
kmeans = KMeans(n_clusters=reasonable_k, random_state=42)
kmeans.fit(transformed_df)
labels = kmeans.labels_

In [ ]:
cluster_centers = pd.DataFrame(
    data=kmeans.cluster_centers_, columns=[transformed_df.columns]
)
cluster_centers

- Recall that we have applied imputation and scaling on the dataset. 
- But we would be able to interpret these clusters better if the centers are in the original scale. 
- So let's apply **inverse transformations to get the cluster center values in the original scale**. 

In [ ]:
data = (
    preprocessor.named_transformers_["pipeline"]
    .named_steps["standardscaler"]
    .inverse_transform(cluster_centers[numeric_features])
)

In [ ]:
org_cluster_centers.columns

In [ ]:
org_cluster_centers = pd.DataFrame(data=data, columns=numeric_features)
org_cluster_centers = org_cluster_centers.reindex(
    sorted(org_cluster_centers.columns), axis=1
)
org_cluster_centers

In [ ]:
cluster_labels = {0: "Transactors", 1: "Revolvers", 2: "Low activity", 3: "VIP/Prime"}
org_cluster_centers["cluster_labels"] = list(cluster_labels.values())

In [ ]:
relevant_cols = [
    "cluster_labels",
    "BALANCE",
    "CREDIT_LIMIT",
    "PRC_FULL_PAYMENT",
    "PURCHASES_FREQUENCY",
    "CASH_ADVANCE",
    "CASH_ADVANCE_FREQUENCY",
    "CASH_ADVANCE_TRX",
]
org_cluster_centers[relevant_cols]

One way to interpret and label the clusters above is as follows. 

#### Transactors
- Credit card users who pay off their balance every month with least amount of interest charges. 
- They are careful with their money. 
- They have lowest balance and cash advance

#### Revolvers
- Credit card users who pay off only part of their monthly balance. They use credit card as a loan.  
- They have highest balance and cash advance, high cash advance frequency, low purchase frequency, high cash advance transactions, low percentage of full payment
- Their credit limit is also high. (Lucrative group for banks 😟.)

#### Low activity
- There is not much activity in the account. It has low balance and not many purchases. 
- Credit card users who have low credit limit.

#### VIP/Prime
- Credit card users who have high credit limit. 
- They have high one-off purchases frequency, high number of purchase transactions. 
- They have high balance but they also have higher percentage of full payment, similar to transactors
- Target for increase credit limit (and increase spending habits)

### More on interpretation of clusters

- In real life, you'll look through all features in detail before assigning meaning to clusters. 
- This is not that easy, especially when you have a large number of features and clusters. 
- One way to approach this would be visualizing the distribution of feature values for each cluster. 
- Some domain knowledge would definitely help at this stage.  

In [ ]:
creditcard_df['cluster'] = labels

Let's check the cluster assignment for the customer who made the maximum cash advance transaction. 

In [ ]:
creditcard_df[creditcard_df["CASH_ADVANCE"] == max_cash_advance] 

What's this client's cluster?

In [ ]:
cluster_labels[creditcard_df[creditcard_df["CASH_ADVANCE"] == max_cash_advance]['cluster'].values[0]]

In [ ]:
def show_hists(df=creditcard_df, cols=["BALANCE", "CASH_ADVANCE"]):
    for i in cols:
        plt.figure(figsize=(35, 5))
        for j in range(4):
            plt.subplot(1, 4, j + 1)
            cluster = df[df["cluster"] == j]
            cluster[i].hist(bins=20)
            plt.title(f"{i}    \nCluster: {cluster_labels[j]} ")

        plt.show()

In [ ]:
show_hists() # Examining clusters for two features. 

In [ ]:
# Uncomment the code below to show histograms for all features. 
# cols = creditcard_df_cluster.columns.to_list()
# cols.remove('CUST_ID')
# cols.remove('cluster')
# show_hists(creditcard_df_cluster, cols)

### Practice exercise for you
- Try out different values for `n_clusters` in `KMeans` and examine the clusters. 
- If you are feeling adventurous, you may try customer segmentation on [All Lending Club loan data](https://www.kaggle.com/wordsforthewise/lending-club). 

<br><br><br><br>

## Final comments, summary, and reflection

### A comment on initialization

- The initialization of K-Means is stochastic, can this affect the results?
    - Answer: ???
- Let's  look at an example. 

In [ ]:
X, y = make_blobs(n_samples=20, centers=3, n_features=2, random_state=10)
mglearn.discrete_scatter(X[:, 0], X[:, 1], markers="o");

In [ ]:
k = 3
n_examples = X.shape[0]
toy_df = pd.DataFrame(X, columns=["feat1", "feat2"])

### Example: Bad initialization 

In [ ]:
np.random.seed(seed=10)
centroids_idx = np.random.choice(range(0, n_examples), size=k)
centroids = X[centroids_idx]

In [ ]:
plot_iterative(toy_df, 6, 4, centroids)

### Example: Better initialization 
The following initialization seems much better. 

In [ ]:
np.random.seed(seed=2)
centroids_idx = np.random.choice(range(0, n_examples), size=k)
centroids = X[centroids_idx]

In [ ]:
plot_iterative(toy_df, 6, 4, centroids)

### What can we do about it?

- One strategy is to **re-run the algorithm several times**. 
    - Check out `n_init` parameter of [`sklearn`'s `KMeans`](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html). 
    - **Best output will be chosen based on Inertia**
- Is it possible to pick `K` in a smart way? 
    - Yes! We can use the so-called [K-Means++](http://ilpubs.stanford.edu:8090/778/1/2006-13.pdf).
    - Intuitively, it picks the initial centroids which are far away from each other. 
    - In other words, K-Means++ gives more chance to select points that are far away from centroids already picked.    
    - By default `sklearn` uses this strategy for initialization. 

### (Optional) Time complexity of K-Means

- Naive implementation of K-Means requires you to compute the distances from all data points to all cluster centers. 
- So there are many distance calculations per iteration. 
- calculating assigning observations to centers is heavy: $\mathcal{O(ndk)}$
- updating centers is light(er): $\mathcal{O(nd)}$

where, 

- $n \rightarrow$ number of examples
- $d \rightarrow$ number of features
- $k \rightarrow$ number of clusters


### Important points to remember

- Clustering is a common unsupervised approach to identify underlying structure in data and grouping points based on similarity. 
- K-Means is a popular clustering algorithm. 

### Important points to remember

**K-Means**
- It requires us to **specify the number of clusters** in advance. 
- Each example is assigned to one (and only one) cluster.
- The labels provided by the algorithm have no actual meaning. 
- The centroids live in the same space as of the dataset but they are **not** actual data points, but instead are average points.
- **It always converges to at least a local minima**. 
  - but its convergence to a global minima is not guaranteed
- Convergence is dependent upon the initial centers and it may converge to a sub-optimal solution. 

### Important points to remember

- Two ways to provide insight into how many clusters are reasonable for the give problem are: **the Elbow method** and **the Silhouette method**.  
- Some applications of K-Means clustering include data exploration, feature engineering, customer segmentation, and document clustering. 
- It takes fair amount of manual effort and domain knowledge to interpret clusters returned by K-Means. 

<br><br><br><br>

## Resources 
- ["Spaghetti Sauce" talk by Malcom Gladwell](https://www.ted.com/talks/malcolm_gladwell_on_spaghetti_sauce?language=en)
- [Visualizing-k-means-clustering](https://www.naftaliharris.com/blog/visualizing-k-means-clustering/) 
- [Visualizing K-Means algorithm with D3.js](http://tech.nitoyon.com/en/blog/2013/11/07/k-means/)
- [Clustering with Scikit with GIFs](https://dashee87.github.io/data%20science/general/Clustering-with-Scikit-with-GIFs/)
- [`sklearn` clustering documentation](https://scikit-learn.org/stable/modules/clustering.html)